# Code

In [42]:
# import libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Signal class
class Signal:
    def __init__(self, sourcedatenam, source, name, signaldate, expirydate, pricelast, pricedata):
        self.sourcedatenam = sourcedatenam
        self.source = source
        self.name = name
        self.signaldate = signaldate
        self.expirydate = expirydate

        self.pricelast = pricelast
        self.pricedata = iter(pricedata)

        self.priceprev = next(self.pricedata, None)
        self.pricenow = next(self.pricedata, None)
        self.pricenext = next(self.pricedata, None)

        self.age = 0
        self.growth = 0
        self.value = 0

        self.buydate = signaldate + timedelta(days=5)
        self.buyprice = self.pricenow

    # function to print position details
    def display(self):
        print(self.sourcedatenam, '   pricelast:', str(self.pricelast), '   priceprev/now/next:', str(self.priceprev), str(self.pricenow), str(self.pricenext), '   growth:', str(self.growth), '   val:', str(self.value))

    # function to add one to age
    def ageOneDay(self):
        self.age += 1

    # function to set pricenow based on value
    def setPrice(self, value):
        self.pricenow = value

    # function to set value based on value
    def setVal(self, value):
        self.value = value

    def updateMonthlyPrice(self):
        self.priceprev = self.pricenow
        self.pricenow = self.pricenext
        self.pricenext = next(self.pricedata, None)

    # function to recalculate value of the position
    def updateMonthlyVal(self):
        self.growth = (self.pricenow - self.priceprev) / self.priceprev
        self.value *= (1 + self.growth)

# Portfolio class (contains signals)
class Portfolio:
    def __init__(self, cash):
        self.cash = cash
        self.signallist = []
        self.solddf = []

    # print each signal in portfolio
    def display(self):
        for signal in self.signallist:
            signal.display()

    # return the total value of portofolio (cash included)
    def getTotalValue(self):
        signal_values = sum(signal.value for signal in self.signallist)
        return signal_values + self.cash
    
    # return the number of signals in portfolio
    def getSize(self):
        return len(self.signallist)
    
    # return the age of oldest signal
    def getMaxAge(self):
        if not self.signallist:
            return None
        ages = np.array([signal.age for signal in self.signallist])
        ages_max = np.max(ages)
        return  ages_max
    
    # return the age of oldest signal
    def getAvgAge(self):
        if not self.signallist:
            return None
        ages_sum = np.array([signal.age for signal in self.signallist])
        ages_avg = np.mean(ages_sum)
        return ages_avg
    
    # function to buy signal
    def buySignal(self, newsig):
        
        if newsig.value > self.getTotalValue():
            raise ValueError('Not enough funds!')

        if self.cash < newsig.value:
            cash_needed = newsig.value - self.cash
            self.cash = 0
            idx = 0

            while cash_needed > 0 and idx < len(self.signallist):
                cash_to_use = min(cash_needed, self.signallist[idx].value)
                self.signallist[idx].value -= cash_to_use
                cash_needed -= cash_to_use
                self.solddf.append([
                    self.signallist[idx].buydate,
                    self.signallist[idx].buyprice,
                    self.signallist[idx].pricenow,
                    cash_to_use
                ])
                if self.signallist[idx].value <= 0:
                    self.signallist.pop(idx)
                else:
                    idx += 1

            self.cash -= cash_needed

        else:
            self.cash -= newsig.value

        self.signallist.append(newsig)

    # function to buy each position in the datafrane
    def buyFromDfRow(self, df_row, value):
        sourcedatenam = df_row['SourceDateNam']
        source = df_row['Source']
        name = df_row['Name and Ticker']
        signaldate = df_row['Signal Date to Use']
        expirydate = df_row['Last Pricing Date']
        pricelast = df_row['Price on Last Date']
        pricedata = df_row.iloc[10:].values
        newsig = Signal(sourcedatenam, source, name, signaldate, expirydate, pricelast, pricedata)
        newsig.setVal(value)
        self.buySignal(newsig)

    # function to buy each position in the datafrane
    def buyFromDf(self, df, value):
        for index, row in df.iterrows():
            self.buyFromDfRow(row, value)
    
    # function to update all prices of all signals
    def updateMonthlyPrice(self):
        for signal in self.signallist:
            signal.updateMonthlyPrice()

    # function to update all values of all signals
    def updateMonthlyVal(self):
        for signal in self.signallist:
            signal.updateMonthlyVal()

    # function to check for positions that lost pricing data during the month
    def dumpExpired(self):
        for signal in self.signallist:
            if signal.pricenext == 0:
                self.cash += signal.value

                # record a sale
                self.solddf.append([signal.buydate, signal.buyprice, signal.pricenow, signal.value])
                
                signal.value = 0

        # remove sold out stocks from portfolio
        self.signallist = [signal for signal in self.signallist if signal.value > 0]

    # function to add one year to all signals
    def ageOneDay(self):
        for signal in self.signallist:
            signal.ageOneDay()

# Test

In [37]:
df = pd.read_excel(r'C:\Users\MarcoHui\Desktop\PythonProjects\PortfolioSim\new data\SimulatorData 30ATS NEW.xlsx', sheet_name='prices', skiprows=1)

In [43]:
# set up
temp = df[46:49]
my_portfolio = Portfolio(5)

# buy signals
my_portfolio.buyFromDf(temp, 1)

# check that they have been bought
print('\nAFTER FIRST BUY')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))


# update price and value
my_portfolio.updateMonthlyPrice()
my_portfolio.updateMonthlyVal()

# check that price and values have been updated
print('\nAFTER ONE PRICES AND VALUES UPDATE')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))

# update price and value until expiry
for i in range(157):
    my_portfolio.updateMonthlyPrice()
    my_portfolio.updateMonthlyVal()

# check that price and values have been updated
print('\nAFTER MANY PRICES AND VALUES UPDATES')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))

# dump expired
my_portfolio.dumpExpired()

# print sell list
print('\nWHAT WE DUMPED')
print(my_portfolio.solddf)

# check that expired signals have been sold at last available price
print('\nAFTER EXPIRED DUMPED')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))

# buy more signals
temp = df[49:50]
my_portfolio.buyFromDf(temp, 4)

# check that cash is gone and oldest stock has been replaced
print('\nAFTER ANOTHER BUY')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))

# print sell list
print('\nWHAT WE SOLD TO FUND NEW SIGNAL')
print(my_portfolio.solddf)


AFTER FIRST BUY
jna34139817The    pricelast: na    priceprev/now/next: 33.71921 33.83364 33.66199    growth: 0    val: 1
zach72139818thi    pricelast: 10.45    priceprev/now/next: 5.9 5.98 5.65    growth: 0    val: 1
jna34139819GOO    pricelast: na    priceprev/now/next: 8.3515 8.05025 8.12975    growth: 0    val: 1
cash:  2

AFTER ONE PRICES AND VALUES UPDATE
jna34139817The    pricelast: na    priceprev/now/next: 33.83364 33.66199 33.29962    growth: -0.005073353029706518    val: 0.9949266469702935
zach72139818thi    pricelast: 10.45    priceprev/now/next: 5.98 5.65 8.34    growth: -0.05518394648829432    val: 0.9448160535117057
jna34139819GOO    pricelast: na    priceprev/now/next: 8.05025 8.12975 7.87675    growth: 0.009875469705909688    val: 1.0098754697059096
cash:  2

AFTER MANY PRICES AND VALUES UPDATES
jna34139817The    pricelast: na    priceprev/now/next: 39.92712 39.85084 39.85084    growth: -0.0019104808961929666    val: 1.1778466638528993
zach72139818thi    pricelast: 10.